# Sklearn Pipeline for Scoring New Data

In [1]:
import psycopg2
import pickle
import pandas as pd

# Import Data
the dataset is available here: kaggle

In [2]:
## load data from local csv 
#df = pd.read_csv(
#        filepath_or_buffer = '../data/bank_customers_churn_dataset.csv',
#        index_col='customer_id'
#)
#
## drop churn column for segmentation
#df.drop('churn', axis=1, inplace=True)
#
## change type of categorical columns "credit_card" and "active_member"
#df['credit_card'] = df['credit_card'].apply(lambda x: 'yes' if x == 1 else 'no')
#df['active_member'] = df['active_member'].apply(lambda x: 'yes' if x == 1 else 'no')
#
#df.head()

In [3]:
def get_data_from_my_postgre_db(password:str, table_name:str)->pd.DataFrame:
    """connects to marketing_analytics db and returns data from table_name as pandas dataframe .
    inputs: database password and table_name"""
    # Connect to the database
    conn = psycopg2.connect(
        database="marketing_analytics", 
        user="postgres", 
        password=password, 
        host="localhost", 
        port="5432")
    # Create a cursor object
    cur = conn.cursor()
    # Execute a SQL query
    #cur.execute("SELECT * FROM bank_customers_churn_dataset")
    cur.execute("SELECT * FROM " + table_name) 
    # Get the column names from the cursor description
    columns = [desc[0] for desc in cur.description]
    # Fetch the results i.e. values
    results = cur.fetchall()
    # Create a dictionary mapping column names to values
    data = [dict(zip(columns, row)) for row in results]
    # Close the connection
    conn.close()
    # turn dictionary into dataframe
    return pd.DataFrame.from_dict(data)

In [4]:
f = open("../private.txt", "r")
pw = f.read()

In [5]:
# get data from postres db
table_name = "bank_customers_churn_dataset"
df = get_data_from_my_postgre_db(pw, table_name)

# set customer id as index
df.set_index('customer_id', inplace=True)

# drop churn column for segmentation
df.drop('churn', axis=1, inplace=True)

# change data types to numeric
df['tenure'] = df['tenure'].apply(lambda x: int(x))
df['products_number'] = df['products_number'].apply(lambda x: int(x))
df['credit_card'] = df['credit_card'].apply(lambda x: int(x))
df['active_member'] = df['active_member'].apply(lambda x: int(x))

# change type of categorical columns "credit_card" and "active_member"
df['credit_card'] = df['credit_card'].apply(lambda x: 'yes' if x == 1 else 'no')
df['active_member'] = df['active_member'].apply(lambda x: 'yes' if x == 1 else 'no')

# Load Model Artifacts

In [6]:
with open('../artifacts/numeric_imputer.pickle', 'rb') as filename: # trained model to impute missing numeric data
    numeric_imputer = pickle.load(filename)

with open('../artifacts/categorical_imputer.pickle', 'rb') as filename: # trained model to impute missing categorical data
    categorical_imputer = pickle.load(filename) 

with open('../artifacts/rare_encoder.pickle', 'rb') as filename: # trained model to encode rare labels
    rare_encoder = pickle.load(filename)

with open('../artifacts/capper.pickle', 'rb') as filename: # trained model to cap outliers
    capper = pickle.load(filename)   

with open('../artifacts/enc.pickle', 'rb') as filename: # trained one hot encoder
    enc = pickle.load(filename)

with open('../artifacts/model.pickle', 'rb') as filename: # trained random forrest classifier
    model = pickle.load(filename)

# Transform Dataset

In [7]:
# get numeric and categorical columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns = df.select_dtypes(include=numerics).columns.to_list()
categorical_columns = df.select_dtypes(exclude=numerics).columns.to_list()

In [8]:
# impute mising numeric features
df_numeric = pd.DataFrame(
    numeric_imputer.transform(df[numeric_columns]), 
    columns=numeric_columns, 
    index=df.index)

# impute mising categorical features
df_categorical = pd.DataFrame(
    categorical_imputer.transform(df[categorical_columns]), 
    columns=categorical_columns, 
    index=df.index)

# concate numeric and categorical features
df = pd.concat([df_numeric, df_categorical], axis=1)

# remove rare labels
df[categorical_columns] = rare_encoder.transform(df[categorical_columns])

# remove outliers
df[numeric_columns] = capper.transform(df[numeric_columns])

# one hot encoding categorical features
df_cat_hotenc = pd.DataFrame(
    enc.transform(df[categorical_columns]), 
    columns=enc.get_feature_names_out(),
    index=df.index) 

# concate numeric and hot-encoded categorical features
df_hotenc = pd.concat([df[numeric_columns], df_cat_hotenc], axis=1)

# predict cluster
labels = model.predict(df_hotenc)

# add cluster label to df
df['cluster'] = labels

df.head()

,credit_score,age,tenure,balance,products_number,estimated_salary,country,gender,credit_card,active_member,cluster
customer_id,,,,,,,,,,,
15634602,619.0,42.0,2.0,0.00,1.0,101348.88,France,Female,yes,yes,1
15647311,608.0,41.0,1.0,83807.86,1.0,112542.58,Spain,Female,no,yes,2
15619304,502.0,42.0,8.0,159660.80,3.0,113931.57,France,Female,yes,no,3
15701354,699.0,39.0,1.0,0.00,2.0,93826.63,France,Female,no,no,3
15737888,850.0,43.0,2.0,125510.82,1.0,79084.10,Spain,Female,yes,yes,1


In [9]:
# Cluster Counts
df['cluster'].value_counts()

1    2734
0    2509
3    2423
2    1169
4    1165
Name: cluster, dtype: int64

In [10]:
# Cluster Centers
pd.concat([
    df.groupby(by='cluster').mean(),
    df.groupby(by='cluster').aggregate(pd.Series.mode)[categorical_columns] ],
    axis=1)

C:\Users\Sebas\AppData\Local\Temp\ipykernel_22156\2551023627.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(by='cluster').mean(),
C:\Users\Sebas\AppData\Local\Temp\ipykernel_22156\2551023627.py:4: FutureWarning: ['estimated_salary'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df.groupby(by='cluster').aggregate(pd.Series.mode)[categorical_columns] ],


,credit_score,age,tenure,balance,products_number,estimated_salary,country,gender,credit_card,active_member
cluster,,,,,,,,,,
0,651.453567,39.720469,5.009964,119730.116134,1.512869,101113.435102,Germany,Male,yes,no
1,652.849305,39.484752,5.032919,60451.419214,1.528936,98028.231372,France,Male,yes,yes
2,651.337896,39.947621,4.796407,64975.713370,1.529737,101585.407280,France,Male,no,yes
3,646.894759,37.340945,5.054065,63463.576232,1.513989,100982.054408,France,Male,yes,no
4,649.837768,37.677730,5.103004,59616.306635,1.567738,99370.597966,Spain,Male,yes,no
